### CREATING A CHATBOT THAT WILL ONLY ANSWER QUESTIONS FROM A GIVEN PDF WHICH BY THIS CASE IS THE KENYAN CONSTITUTION PDF AND I'LL BE USING RAG

In [1]:
#Import Libraries after installing modules from the terminal
import os
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate

c:\Users\ADEN\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#Set api environment
os.environ["GROQ_API_KEY"]="gsk_5N9pIIIjQtN5k7I..."

In [3]:
#Create paths
DATA_PATH="/content/data"
CHROMA_PATH="/content/chroma_db"

In [4]:
#Load the document/pdf
loader=PyPDFDirectoryLoader("data")
raw_documents = loader.load()
print(len(raw_documents))

193


In [5]:
#Split and chunk
splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=150
)
chunks = splitter.split_documents(raw_documents)
#print(f"total chunks are: {len(chunks)}")


In [ ]:
#Create embeddings
embeddings_model=HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [7]:
#Generate a vector database
vector_store=Chroma(
    collection_name="sample_collection",
    embedding_function=embeddings_model,
    persist_directory=CHROMA_PATH,
)

In [8]:
# 4. Add chunks into DB

vector_store.add_documents(documents=chunks)

print(f"Inserted {len(chunks)} chunks into Chroma")

Inserted 1315 chunks into Chroma


In [9]:
#Retrieving
retriever=vector_store.as_retriever()

In [10]:
#Call in the LLM
llm=ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=0.5
)

In [11]:
#Prompting the document
rag_prompt_template = PromptTemplate(
    input_variables=["question", "history", "knowledge"],
    template="""
    You are an assistant that answers based ONLY on the provided knowledge.
    Do not say "according to documents" or similar.

    Question: {question}

    Conversation history: {history}

    Knowledge:
    {knowledge}
    """
)


In [12]:
#Chaining using a function
def ask_bot(question, history=None):
    history = history or []

    # Retrieve docs
    docs = retriever.invoke(question)
    knowledge = "\n\n".join([doc.page_content for doc in docs])

    # Format the prompt using the template
    rag_prompt = rag_prompt_template.format(
        question=question,
        history=history,
        knowledge=knowledge
    )

    # Collect response (streaming)
    response = ""
    for chunk in llm.stream(rag_prompt):
        response += chunk.content
    return response


In [40]:
#Testing if it'll answer the questions
print(ask_bot("What are the right of an arrested person?"))

An arrested person has the right to be informed promptly, in a language they understand, of:

1. The reason for the arrest.
2. The right to remain silent.
3. The consequences of not remaining silent.

They also have the right to remain silent.


In [14]:
print(ask_bot("About fair hearing"))

There shall be a right to have a fair hearing before a registration is cancelled.


In [15]:
print(ask_bot("What are the main roles of the parliament in this constitution?"))

The main roles of the parliament in this constitution are to derive the legislative authority of the Republic from the people and exercise it at the national level.


In [16]:
print(ask_bot("What does the document potray about the devolved government?"))

The document portrays that the devolved government should have a division of revenue. It also mentions mechanisms to ensure that the Commission on the Implementation of the Constitution can effectively monitor the implementation of the system of devolved government.


In [17]:
print(ask_bot("Tell me about Luxdev"))

There is no information about Luxdev in the provided knowledge.


In [18]:
print(ask_bot("Tell me about kiambu"))

I couldn't find any information about Kiambu in the provided knowledge. However, I noticed that the provided text seems to be a song or a poem, possibly the national anthem of Kenya, given the mention of "Nchi yetu ya Kenya" which translates to "Our Homeland of Kenya".


In [19]:
print(ask_bot("Tell me about Kileleshwa"))

I couldn't find any information about Kileleshwa. The provided knowledge appears to be a patriotic song lyrics in Swahili, but it doesn't mention Kileleshwa.


In [22]:
print(ask_bot("Dictate the Kenyan National Anthem from the source"))

The Kenyan National Anthem is:

Ee Mungu nguvu yetu O God of all creation
Ilete baraka kwetu. Bless this our land and nation.
Haki iwe ngao na mlinzi Justice be our shield and defender
Natukae na undugu May we dwell in unity
Amani na uhuru Peace and liberty
Raha tupate na ustawi Plenty be found within our borders.


In [24]:
print(ask_bot("What are the main roles of the parliament in this constitution?"))

The main roles of the parliament in this constitution are not explicitly stated in the provided knowledge. However, it can be inferred that the legislative authority of the Republic is derived from the people and is vested in and exercised by Parliament.


In [25]:
print(ask_bot("What does article 37 say"))

There is no information provided about article 37. However, there are multiple repetitions of the information about article 42.


In [26]:
print(ask_bot("how many articles are there from this document"))

There are 5 articles in this document.


In [27]:
print(ask_bot("What do the 8 articles entail"))

The 8 articles appear to be repetitive. However, based on the provided text, the common content of the 8 articles can be summarized as follows:

The articles bind all State organs, State officers, public officers, and all persons to:

- Apply or interpret the Constitution
- Enact, apply, or interpret any law
- Make or implement public policy decisions

Additionally, the national values and principles of governance are included, but the specific values and principles are not mentioned in the provided text.


In [28]:
print(ask_bot("What does Article 1 state"))

Article 1 states that it binds all State organs, State officers, public officers, and all persons whenever any of them:

(a) applies or interprets this Constitution;
(b) enacts, applies or interprets any law; or
(c) makes or implements public policy decisions.


In [29]:
print(ask_bot("defence forces"))

There is no specific information provided about the Defence Forces in the given knowledge. However, it mentions that:

1. The Defence Forces can be affected by unrest or instability only with the approval of the National Assembly.
2. The composition of the command of the Defence Forces shall reflect the regional and ethnic diversity of the people of Kenya.
3. A Defence Council has been established, which consists of the Cabinet Secretary responsible for defence as the chairperson.

However, for more information about the Defence Forces, additional context or details are needed.


In [30]:
print(ask_bot("National security"))

National security is the protection against internal and external threats to Kenya's territorial integrity and sovereignty, its people, their rights, freedoms, property, peace, stability, and prosperity, and other national interests.


In [31]:
print(ask_bot("What are the roles of courts in Kenya?"))

The roles of courts in Kenya include those of the subordinate courts, which are:

1. The Magistrates courts
2. The Kadhis' courts
3. The Courts Martial
4. Any other court or local tribunal established by an Act of Parliament, other than the courts established as required by Article 162 (2).

These subordinate courts have specific roles and jurisdictions, but their exact roles are not specified in the provided knowledge.


In [32]:
print(ask_bot("What are the roles of the high court of Kenya?"))

The High Court of Kenya is established by the Constitution of Kenya, 2010. However, the provided knowledge does not explicitly state the roles of the High Court.


In [49]:
print(ask_bot("What does article 165 state"))

Article 165 states: 

(2) If, with respect to any particular matter—
(a) a law that was in effect immediately before the effective date assigns responsibility for that matter to a particular State.


In [34]:
print(ask_bot("What does the document potray about the devolved government?"))

The document portrays that the devolved government should have a division of revenue. It also mentions that the Commission on the Implementation of the Constitution should be able to perform its role in monitoring the implementation of the system of devolved government effectively.


In [35]:
print(ask_bot("What does the document state about state religion"))

The document states that there shall be no State religion.


In [36]:
print(ask_bot("tell me about a cat"))

There is no direct information about cats in the provided knowledge. However, it mentions "facilities for the accommodation, care and burial of animals" under Animal control and welfare in the Constitution of Kenya, 2010. This implies that the government of Kenya has laws and regulations to protect and care for animals, which would likely include cats.


In [37]:
print(ask_bot("Tell me about Luxdev"))

I couldn't find any information about Luxdev in the provided knowledge.


In [38]:
print(ask_bot("Tell me about Icefloqx"))

I couldn't find any information about Icefloqx in my knowledge.
